In [5]:
from numpy import dot
from numpy.linalg import norm
import pandas as pd
import numpy as np

In [6]:
merged = pd.read_pickle("../Data/merged.pkl")

In [7]:
data = pd.read_excel('../Data/ИтоговыеДанные.xlsx')

In [8]:
data

,Unnamed: 0,Торговый день,ID Клиента,Регион проживания,МАГАЗИН,"Тип карточки (B-VISA, S-MASTERCARD или МИР)",Сумма расходов,Предоставленная скидка,Возраст клиента
0,0,2020-01-08,10221462,Московская область,"Ресторан ""Буйвол""",B,66816,11,31
1,1,2020-01-20,10220237,Красноярский край,Барбершоп,B,2538,9,33
2,2,2020-01-20,10215286,Московская область,Такси,B,6241,6,36
3,3,2020-01-20,10220237,Красноярский край,"Бар ""Коктейль""",B,2802,9,33
4,4,2020-01-24,11128769,г.Москва,Барбершоп,B,7227,18,24
...,...,...,...,...,...,...,...,...,...
67785,9860,2022-12-29,10723660,г.Москва,Барбершоп,B,163,7,35
67786,9861,2022-12-30,10724982,Челябинская область,Барбершоп,S,848,15,27
67787,9862,2022-12-30,10162467,Республика Северная Осетия-Алания,Барбершоп,S,12683,0,39
67788,9863,2022-12-30,11125776,г.Москва,Покупка техники,B,3554,20,22


##  Select the Target Group

#### Which sector would you like to improve?
By choosing, for example, the Gastronomic sector, you select a group of customers who show high attendance and spendings, having a discount above 15%.
Further, the recommender system looks for similar customers who can be offered discounts for the selected sector and the indicators will increase.

In [38]:
# # Attendance
# ideals = [list(map(float,merged.loc[i].to_numpy(dtype=float))) for i in 
#          [11018820, 11127430, 11259771, 11260095]]

In [33]:
# # Spendings
# ideals = [list(map(float,merged.loc[i].to_numpy(dtype=float))) for i in 
#           [10574034, 11018820, 11127430, 11259771, 11260095]]

In [14]:
# # Sector: Gastronomic
# ideals = [list(map(float,merged.loc[i].to_numpy(dtype=float))) for i in 
#           [11127430, 10355641, 11259771, 11018820, 11260095, 10356174]]

In [28]:
# # Sector: Tourism
# ideals = [list(map(float,merged.loc[i].to_numpy(dtype=float))) for i in 
#           [11127430, 10355641, 10562323, 11259771, 11018820, 11260095]]

In [43]:
# # Sector: Shopping
# ideals = [list(map(float,merged.loc[i].to_numpy(dtype=float))) for i in 
#     [11127430, 10355641, 11259771, 11018820, 10359664, 10409000, 10830303, 10216655, 10201728]]

In [48]:
# # Sector: Auto
# ideals = [list(map(float,merged.loc[i].to_numpy(dtype=float))) for i in 
#           [11127430, 10355641, 11128132, 11259771, 11018820, 10830303]]

In [53]:
# # Sector: Beauty
# ideals = [list(map(float,merged.loc[i].to_numpy(dtype=float))) for i in 
#           [11127430, 10355641, 10574034, 11259771, 11018820, 10358256]]

In [58]:
# Sector: Other
ideals = [list(map(float,merged.loc[i].to_numpy(dtype=float))) for i in 
          [11127430, 10355641, 11259771, 11018820, 10356174]]

In [20]:
merge_vec = list(map(list,merged.to_numpy(dtype=float)))

In [21]:
clients = {key:val for val,key in zip(merge_vec,merged.index.values.tolist())}

In [59]:
class Recommender:
    def __init__(self, ideal):
        self.clients_vector = None
        self.ideal_clients = ideal
          
    def cos_sim(self,a,b):
        similarity = dot(a, b)/(norm(a)*norm(b))
        return similarity
    
    def collaborative_filtering(self,user):
        count = 0
        for i in self.ideal_clients:
            similarity = self.cos_sim(i,user)
            if 0.9 <= similarity < 1:
                count += 1
        return count >= 2

    def recommend_clients(self, clients):
        valid_clients = []
        for client_id, client in clients.items():
            if self.collaborative_filtering(client):
                valid_clients.append(client_id)
        return valid_clients



In [60]:
recommender = Recommender(ideals)
Clients = recommender.recommend_clients(clients)

### The recommender system brought out the customers most susceptible to take advantage of the proposed large discount, thereby increasing the attendance and revenue indicators for the selected sector.

In [61]:
print(len(Clients))
Clients

21


[10216816,
 10217384,
 10218209,
 10218864,
 10219380,
 10354725,
 10355392,
 10361741,
 10402701,
 10404735,
 10574677,
 10583558,
 10837991,
 11008961,
 11011968,
 11012574,
 11018939,
 11135326,
 11257217,
 11259088,
 11259804]

In [25]:
# Check the client attendance
merged.loc[10216816]

Автотранспорт                 0.000000
Алкогольный магазин          85.000000
Бар "Иной"                    0.000000
Бар "Коктейль"              183.000000
Барбершоп                   295.000000
Не определено               125.000000
Перевод денег на депозит      0.000000
Покупка одежды               24.000000
Покупка техники              40.000000
Покупки Ozon                 26.000000
Ресторан "Буйвол"           169.000000
Салон красоты                 0.000000
Такси                        76.000000
Турагенство                  83.000000
Сумма расходов               -0.261918
Name: 10216816, dtype: float64

In [62]:
for i in Clients:
    print('<br> ID: ',i,'|', 'Age: ', data[data['ID Клиента'] == i]['Возраст клиента'].unique()[0],
         '|','Region: ', data[data['ID Клиента'] == i]['Регион проживания'].unique()[0],
         '|','Current Discount: ', data[data['ID Клиента'] == i]['Предоставленная скидка'].unique()[0],'<br>')

<br> ID:  10216816 | Age:  32 | Region:  Белгородская область | Current Discount:  10 <br>
<br> ID:  10217384 | Age:  41 | Region:  Пензенская область | Current Discount:  0 <br>
<br> ID:  10218209 | Age:  28 | Region:  Томская область | Current Discount:  14 <br>
<br> ID:  10218864 | Age:  35 | Region:  г.Санкт-Петербург | Current Discount:  7 <br>
<br> ID:  10219380 | Age:  39 | Region:  Калининградская область | Current Discount:  0 <br>
<br> ID:  10354725 | Age:  37 | Region:  Челябинская область | Current Discount:  5 <br>
<br> ID:  10355392 | Age:  39 | Region:  Республика Башкортостан | Current Discount:  0 <br>
<br> ID:  10361741 | Age:  20 | Region:  Алтайский край | Current Discount:  22 <br>
<br> ID:  10402701 | Age:  34 | Region:  Московская область | Current Discount:  8 <br>
<br> ID:  10404735 | Age:  34 | Region:  Волгоградская область | Current Discount:  8 <br>
<br> ID:  10574677 | Age:  28 | Region:  Челябинская область | Current Discount:  14 <br>
<br> ID:  10583558 